In [ ]:
import sys
print(f"Python version: {sys.version}")

!pip install torch-geometric -q
!pip install deepchem -q

print("\nInstalling RDKit...")
!pip install rdkit

print("\n  All packages installed!")

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.4/552.4 kB 9.9 MB/s eta 0:00:00

Installing RDKit...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 20.6 MB/s eta 0:00:00

  All packages installed!


In [ ]:
# Force CPU mode
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

print(" Switched to CPU mode")
print(" Expected training time: ~2 hours for 30 epochs")

🔄 Switched to CPU mode
⏱️ Expected training time: ~2 hours for 30 epochs


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data, Batch
import deepchem as dc
import math
import time

print(" All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"DeepChem version: {dc.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


 All imports successful!
PyTorch version: 2.9.0+cpu
DeepChem version: 2.5.0
GPU available: False


In [ ]:
class RadialBasis(nn.Module):
    """Bessel radial basis for distance encoding"""

    def __init__(self, num_basis=8, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        self.frequencies = nn.Parameter(
            torch.arange(1, num_basis + 1) * math.pi / cutoff,
            requires_grad=False
        )

    def forward(self, distances):
        """Encode distances as basis functions"""
        # Cutoff envelope
        envelope = torch.where(
            distances < self.cutoff,
            torch.cos(distances * math.pi / (2 * self.cutoff)) ** 2,
            torch.zeros_like(distances)
        )

        # Bessel basis
        d = distances.unsqueeze(-1)
        basis = torch.sin(self.frequencies * d) / d

        return basis * envelope.unsqueeze(-1)

print(" RadialBasis defined")

 RadialBasis defined


In [ ]:
class MACEInteraction(MessagePassing):
    """MACE message passing interaction layer"""

    def __init__(self, hidden_dim, num_basis):
        super().__init__(aggr='add')

        # Message network
        self.message_net = nn.Sequential(
            nn.Linear(hidden_dim * 2 + num_basis, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Update network
        self.update_net = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        """Message passing step"""
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr)
        out = self.update_net(torch.cat([x, out], dim=-1))
        return out

    def message(self, x_i, x_j, edge_attr):
        """Construct messages"""
        msg_input = torch.cat([x_i, x_j, edge_attr], dim=-1)
        return self.message_net(msg_input)

print(" MACEInteraction defined")

 MACEInteraction defined


In [ ]:
class MACE(nn.Module):
    """MACE: Multi-Atomic Cluster Expansion Neural Network"""

    def __init__(
        self,
        num_elements=100,
        hidden_dim=64,
        num_interactions=2,
        num_basis=8,
        cutoff=5.0
    ):
        super().__init__()

        self.cutoff = cutoff

        # Atom embedding
        self.atom_embedding = nn.Embedding(num_elements, hidden_dim)

        # Radial basis
        self.radial_basis = RadialBasis(num_basis, cutoff)

        # Interaction layers
        self.interactions = nn.ModuleList([
            MACEInteraction(hidden_dim, num_basis)
            for _ in range(num_interactions)
        ])

        # Energy head
        self.energy_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, z, pos, edge_index, batch=None, num_graphs=None):
        """
        Forward pass

        Args:
            z: (N,) atomic numbers
            pos: (N, 3) atomic positions
            edge_index: (2, E) edge indices
            batch: (N,) batch assignment
            num_graphs: (int) explicit number of graphs in the batch, used for scatter's dim_size

        Returns:
            energy: (batch_size,) or scalar
            forces: (N, 3) atomic forces
        """
        # Edge features
        row, col = edge_index
        edge_vec = pos[row] - pos[col]
        edge_dist = edge_vec.norm(dim=-1)
        edge_attr = self.radial_basis(edge_dist)

        # Embed atoms
        x = self.atom_embedding(z)

        # Message passing with residual connections
        for interaction in self.interactions:
            x = x + interaction(x, edge_index, edge_attr)

        # Predict atomic energies
        atomic_energies = self.energy_head(x)

        # Sum to molecular energy
        if batch is None:
            energy = atomic_energies.sum()
        else:
            from torch_geometric.utils import scatter
            if num_graphs is not None:
                energy = scatter(atomic_energies, batch, dim=0, reduce='sum', dim_size=num_graphs)
            else:
                # Fallback to inference if num_graphs is not provided (e.g., during testing with single molecule)
                energy = scatter(atomic_energies, batch, dim=0, reduce='sum')

        energy = energy.squeeze(-1)

        forces = None  # Just return None, don't calculate

        return energy, forces

print(" MACE neural network defined")

 MACE neural network defined


In [ ]:
from deepchem.models.losses import Loss

class MACELoss(Loss):
    """Loss function for MACE"""

    def __init__(self, energy_weight=1.0):
        super().__init__()
        self.energy_weight = energy_weight

    def _create_pytorch_loss(self):
        """Required by DeepChem - return a PyTorch loss function"""
        # Return nn.MSELoss directly
        return torch.nn.MSELoss()

print(" MACELoss defined (using PyTorch MSELoss)")

 MACELoss defined (using PyTorch MSELoss)


In [ ]:
from deepchem.models import TorchModel
import torch
import torch.nn as nn
import numpy as np
from torch_geometric.data import Data, Batch
from rdkit import Chem


class MACEWrapper(nn.Module):
    def __init__(self, mace_net):
        super().__init__()
        self.mace = mace_net

    def forward(self, inputs):
        if isinstance(inputs, list):
            pyg_batch = inputs[0]
        else:
            pyg_batch = inputs

        energy, _ = self.mace(
            z=pyg_batch.z,
            pos=pyg_batch.pos,
            edge_index=pyg_batch.edge_index,
            batch=pyg_batch.batch
        )
        return energy


class SimpleMACEModel(TorchModel):
    def __init__(self, coords_cache, **kwargs):
        device = torch.device("cpu")  # Force CPU

        self.coords_cache = coords_cache
        kwargs.setdefault("batch_size", 4)
        kwargs.setdefault("learning_rate", 0.001)

        self._internal_batch_size = kwargs["batch_size"]
        self.learning_rate = kwargs["learning_rate"]

        mace_net = MACE(hidden_dim=64, num_interactions=3)
        wrapper = MACEWrapper(mace_net)
        kwargs["device"] = device

        super().__init__(
            model=wrapper,
            loss=MACELoss(),
            output_types=["prediction"],
            n_tasks=1,
            **kwargs
        )

        self._pytorch_optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.learning_rate
        )

    def _prepare_batch(self, batch):
        if len(batch) >= 2:
            inputs, labels = batch[0], batch[1]
            weights = batch[2] if len(batch) > 2 else None
            ids = batch[3] if len(batch) > 3 else None
        else:
            inputs = batch[0]
            labels = None
            weights = None
            ids = None

        if labels is not None:
            y = labels[0] if isinstance(labels, list) else labels
            if hasattr(y, "shape") and len(y.shape) > 1:
                y = y[:, 0]
        else:
            y = None

        X = inputs[0] if isinstance(inputs, list) else inputs
        device = next(self.model.parameters()).device

        data_list = []

        for i in range(len(X)):
            graph = X[i]
            num_atoms = graph.get_num_atoms()

            smiles = ids[i] if ids is not None else None

            if smiles:
                mol = Chem.MolFromSmiles(smiles)
                if mol is not None:
                    rdkit_atomic_nums = [atom.GetAtomicNum() for atom in mol.GetAtoms()]

                    if len(rdkit_atomic_nums) == num_atoms:
                        raw_atomic_nums = np.array(rdkit_atomic_nums)
                    elif len(rdkit_atomic_nums) < num_atoms:
                        padding = [1] * (num_atoms - len(rdkit_atomic_nums))
                        raw_atomic_nums = np.array(rdkit_atomic_nums + padding)
                    else:
                        raw_atomic_nums = np.array(rdkit_atomic_nums[:num_atoms])
                else:
                    raw_atomic_nums = np.full(num_atoms, 6, dtype=np.int64)
            else:
                raw_atomic_nums = np.full(num_atoms, 6, dtype=np.int64)

            atomic_nums = np.clip(raw_atomic_nums, 1, 100).astype(np.int64)
            z_indices = atomic_nums - 1
            z = torch.tensor(z_indices, dtype=torch.long, device=device)
            z = torch.clamp(z, min=0, max=99)

            if ids is not None and ids[i] in self.coords_cache:
                coords = self.coords_cache[ids[i]]

                if coords.shape[0] == num_atoms:
                    pos = torch.tensor(coords, dtype=torch.float32, device=device)
                elif coords.shape[0] < num_atoms:
                    padding_size = num_atoms - coords.shape[0]
                    last_pos = coords[-1] if len(coords) > 0 else np.array([0, 0, 0])
                    padding = last_pos + np.random.randn(padding_size, 3) * 0.1
                    coords_padded = np.vstack([coords, padding])
                    pos = torch.tensor(coords_padded, dtype=torch.float32, device=device)
                else:
                    pos = torch.tensor(coords[:num_atoms], dtype=torch.float32, device=device)
            else:
                pos = torch.randn(num_atoms, 3, device=device)

            edge_index = build_distance_edges(pos, cutoff=5.0).to(device)

            if y is not None:
                y_val = torch.tensor([y[i]], dtype=torch.float32, device=device)
            else:
                y_val = torch.zeros(1, dtype=torch.float32, device=device)

            data_list.append(Data(z=z, pos=pos, edge_index=edge_index, y=y_val))

        pyg_batch = Batch.from_data_list(data_list)

        if weights is None:
            weights = [torch.ones(len(X), dtype=torch.float32, device=device)]
        elif isinstance(weights, np.ndarray):
            weights = [torch.tensor(weights, dtype=torch.float32, device=device)]

        return ([pyg_batch], [pyg_batch.y], weights)

    def predict(self, dataset, transformers=[], output_types=None):
        all_predictions = []
        for batch in dataset.iterbatches(
            batch_size=self._internal_batch_size,
            deterministic=True
        ):
            self.model.eval()
            with torch.no_grad():
                inputs, _, _ = self._prepare_batch((batch[0], None, None, None))
                outputs = self.model(inputs)
                if isinstance(outputs, torch.Tensor):
                    outputs = outputs.cpu().numpy()
                all_predictions.append(outputs)
        return np.concatenate(all_predictions)

    def fit(self, dataset, nb_epoch=10, **kwargs):
        import time
        print(" Using CUSTOM fit method (CPU mode)")

        for epoch in range(nb_epoch):
            start = time.time()
            total_loss = 0.0
            n_batches = 0

            self.model.train()

            for batch in dataset.iterbatches(
                batch_size=self._internal_batch_size,
                deterministic=True
            ):
                inputs, labels, _ = self._prepare_batch(batch)
                self._pytorch_optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = torch.nn.functional.mse_loss(outputs, labels[0])
                loss.backward()
                self._pytorch_optimizer.step()
                total_loss += loss.item()
                n_batches += 1

            avg_loss = total_loss / n_batches
            elapsed = time.time() - start

            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(
                    f"Epoch {epoch+1}/{nb_epoch}, "
                    f"Loss: {avg_loss:.4f}, "
                    f"Time: {elapsed:.1f}s"
                )

        print(" Training complete")
        return avg_loss


print("✅ SimpleMACEModel - FIXED atom count matching + CPU mode!")


✅ SimpleMACEModel - FIXED atom count matching + CPU mode!


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def get_molecule_coordinates(smiles):
  """
  what this does converts smiles strings into 3d coordinates
  steps:
  parse smiles into molecule object
  generate 3d strcuture using force fields
  optimize geometry (minimize energy)
  extract xyz coordinates
  """
  mol = Chem.MolFromSmiles(smiles)
  if mol is None:
    return None

    #add missing h atoms for smiles
  mol = Chem.AddHs(mol)

  #embedded in 3d space
  # Check for embedding success
  if AllChem.EmbedMolecule(mol, randomSeed=42) == -1: # Returns -1 if embedding fails
      return None

  # Ensure a conformer was actually generated
  if mol.GetNumConformers() == 0:
      return None

  #energy optimize
  AllChem.MMFFOptimizeMolecule(mol)

  #extract coordinates
  conf =mol.GetConformer()
  coords =np.array([
      [
          conf.GetAtomPosition(i).x,
          conf.GetAtomPosition(i).y,
          conf.GetAtomPosition(i).z
      ]
      for i in range(mol.GetNumAtoms())



  ])
  return coords

  #test it
print("\n Testing on water (h20)...")
water_coords =get_molecule_coordinates("O")
print(f" water coordinates shape: {water_coords.shape}")
print(f" coordinates (angstorms): \n{water_coords}")

print("\n testingon methane (ch4)")
methane_coords =get_molecule_coordinates('C')
print(f" Methane coordinates shape: {methane_coords.shape}")
print(f" First 3 atoms: \n{methane_coords[:3]}")


 Testing on water (h20)...
 water coordinates shape: (3, 3)
 coordinates (angstorms): 
[[ 0.00754405  0.39774343  0.        ]
 [-0.76710314 -0.18439316  0.        ]
 [ 0.75955908 -0.21335027  0.        ]]

 testingon methane (ch4)
 Methane coordinates shape: (5, 3)
 First 3 atoms: 
[[-1.66630037e-08 -7.37058382e-09 -1.93574361e-08]
 [-6.75342522e-01  8.54122609e-01 -8.53613529e-02]
 [-3.94946251e-01 -8.35937931e-01 -5.81484974e-01]]


In [ ]:
print("="*70)
print("TESTING MACE ON H2O MOLECULE")
print("="*70)

# Create water molecule
z = torch.tensor([8, 1, 1], dtype=torch.long)  # O, H, H
pos = torch.tensor(
    [[0.0, 0.0, 0.0],
     [0.96, 0.0, 0.0],
     [-0.24, 0.93, 0.0]], requires_grad=True)

edge_index = torch.tensor(
    [[0, 1, 0, 2, 1, 2],
     [1, 0, 2, 0, 2, 1]], dtype=torch.long)

# Create MACE
mace = MACE(hidden_dim=32, num_interactions=2)

# Forward pass
energy, forces = mace(z, pos, edge_index)

print(f"\n  MACE works!")
print(f"   Input: H2O (3 atoms)")
print(f"   Energy: {energy.item():.6f}")

print("\n  MACE neural network tested successfully!")

TESTING MACE ON H2O MOLECULE

  MACE works!
   Input: H2O (3 atoms)
   Energy: -0.554432

  MACE neural network tested successfully!


In [ ]:
def build_distance_edges(pos, cutoff=5.0):
    """Build edges based on distance cutoff
    Physics rule: only connect atoms if distance < cutoff

    Args:
        pos: (N, 3) tensor of atomic positions in angstroms

    Returns:
        edge_index: (2, E) tensor of edges
    """
    num_atoms = pos.shape[0]
    edge_index = []

    # Check distance between every pair of atoms
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                dist = torch.norm(pos[i] - pos[j])
                # Only add edge if within cutoff
                if dist < cutoff:
                    edge_index.append([i, j])

    # Safety check: If NO edges found
    if not edge_index:
        print(f"⚠️ Warning: No edges within {cutoff}Å cutoff!")
        print(f"   Connecting each atom to nearest neighbor as fallback")

        for i in range(num_atoms):
            distances = []
            for j in range(num_atoms):
                if j != i:
                    distances.append(torch.norm(pos[i] - pos[j]))

            nearest_idx = torch.argmin(torch.stack(distances))
            edge_index.append([i, nearest_idx.item()])

    # Convert to tensor format
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    return edge_index

print("✅ build_distance_edges function created!")

✅ build_distance_edges function created!


In [ ]:
# test the distance function
print(" Testing distance based edges")

def build_distance_edges(pos, cutoff=5.0):
  """ build edges based on distance cutoff
  physics rule: only connect atoms if distance < cutoff

  args:
  pos:(N 3 ) tensor of atomic positions in angstroms

  returns:
  edge_index:(2, E ) tensor of edges

  """
  num_atoms =pos.shape[0]
  edge_index =[]

  # what this does: check distance between every pair of atoms

  for i in range(num_atoms):
    for j in range(num_atoms):
       if i!=j:
        # FIX: Remove list wrapper around tensor in torch.norm
        dist=torch.norm(pos[i] -pos[j])
        #only add edge if within cutoff
        if dist < cutoff:
          edge_index.append([i,j])

  # Safety check: If NO edges found
  if not edge_index:
      print(f"⚠️ Warning: No edges within {cutoff}Å cutoff!")
      print(f"   Connecting each atom to nearest neighbor as fallback")

      for i in range(num_atoms):
          distances = []
          for j in range(num_atoms):
              if j != i:
                  distances.append(torch.norm(pos[i] - pos[j]))

          if distances: # Ensure distances is not empty for single-atom cases
              nearest_idx = torch.argmin(torch.stack(distances))
              # Original code had a bug here, mapping to nearest_idx.item() which is only relative to remaining j's
              # A more robust fallback would be to connect to the atom with min distance index
              # For simplicity and to match the original intent, will use the nearest_idx from the distances list.
              # This is still a bit problematic if the original graph has 0 edges and only 1 atom, distances would be empty
              # However, for 2+ atoms, this will work. Let's assume multi-atom molecules for now.
              # Re-calculating the true index:
              min_dist = float('inf')
              true_nearest_j = -1
              for potential_j in range(num_atoms):
                  if potential_j != i:
                      current_dist = torch.norm(pos[i] - pos[potential_j])
                      if current_dist < min_dist:
                          min_dist = current_dist
                          true_nearest_j = potential_j
              if true_nearest_j != -1:
                  edge_index.append([i, true_nearest_j])

  # Convert to tensor format (FIX: This line MUST be outside the outer loop)
  edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

  return edge_index  # FIX: This line MUST be outside the outer loop

print("\n1. test: linear molecule (3 atoms in a row )")

pos_linear =torch.tensor([
    [0.0, 0.0 ,0.0],
    [2.0, 0.0,0.0],
    [4.0,0.0,0.0]

], dtype=torch.float32)

edges_5A =build_distance_edges(pos_linear ,cutoff=5.0)
edges_3A =build_distance_edges(pos_linear ,cutoff=3.0)

print(f" positions: 3 atoms at 0Å, 2Å, 4Å")
print(f" 5Å cutoff edges: {edges_5A.shape[1]} edges")
print(f" Edges:{edges_5A.t().tolist()}")
print(f" expected: all pairs connected (0-1, 1-0 ,0-2,2-0,1-2,2-1)")
print(f" 3Å cutoff edges: {edges_3A.shape[1]} edges")
print(f" Edges:{edges_3A.t().tolist()}")
print(f" Expected:only neighbours (0-1, 1-0, 1-2, 2-1)")

# spread out molecule
print("\n2 Test: spread out molecule")
pos_spread =torch.tensor([
    [0.0, 0.0 ,0.0],
    [10.0, 0.0,0.0],
    [20.0,0.0,0.0]

],dtype=torch.float32)
edges_5A =build_distance_edges(pos_spread ,cutoff=5.0)
print(f" positions:atoms at 0Å, 10Å, 20Å")
print(f" with 5.0Å cutoff: {edges_5A.shape[1]} edges")
print(f" should trigger fallback warning")

#test 3 real water molecule

pos_water =torch.tensor([
     [0.0, 0.0, 0.0],      # Oxygen
    [0.96, 0.0, 0.0],     # Hydrogen 1 (O-H bond ~1Å)
    [-0.24, 0.93, 0.0]    # Hydrogen 2
], dtype=torch.float32)

edges_water =build_distance_edges(pos_water, cutoff=5.0)
print(f" water molecule(O +2H)")
print(f" edges: {edges_water.shape[1]}")
print(f" edges:{edges_water.t().tolist()}")

print("\n distance based edge construction tested!!")


 Testing distance based edges

1. test: linear molecule (3 atoms in a row )
 positions: 3 atoms at 0Å, 2Å, 4Å
 5Å cutoff edges: 6 edges
 Edges:[[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]
 expected: all pairs connected (0-1, 1-0 ,0-2,2-0,1-2,2-1)
 3Å cutoff edges: 4 edges
 Edges:[[0, 1], [1, 0], [1, 2], [2, 1]]
 Expected:only neighbours (0-1, 1-0, 1-2, 2-1)

2 Test: spread out molecule
⚠️ Warning: No edges within 5.0Å cutoff!
   Connecting each atom to nearest neighbor as fallback
 positions:atoms at 0Å, 10Å, 20Å
 with 5.0Å cutoff: 3 edges
 should trigger fallback warning
 water molecule(O +2H)
 edges: 6
 edges:[[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]

 distance based edge construction tested!!


In [ ]:
# test on QM9 Molecule
print("="*70)
print("TESTING MACE ON QM9 MOLECULE")
print("="*70)
import deepchem as dc
# wha does
tasks, datasets,_=dc.molnet.load_qm9(featurizer="graphConv",reload=True)
train =datasets[0]
#get first molecule
test_smiles =train.ids[0]
test_graph =train.X[0]
test_energy =train.y[0 ,0]

print(f"\n1. Molecule from QM9:")
print(f" SMILES:{test_smiles}")
print(f" Number of atoms: {test_graph.get_num_atoms()}")
print(f" true energy:{test_energy:.3f}kcal/mol")

#Extract real coordinates
print(f"\n2.Extracting 3d coordinates...")
coords =get_molecule_coordinates(test_smiles)

if coords is not None:
  print(f" success")
  print(f" Shape:{coords.shape}")
  print(f"Coordinates:\n{coords}")

  #verify it makes sense
  print(f"\n3. Sanity check:")
  print(f" Min coordinates: {coords.min():.2f} Angstroms")
  print(f" Max coordinates: {coords.max():.2f} Angstroms")
  print(f" center of mass: {coords.mean(axis=0)}")

  #check distances
  from scipy.spatial.distance import pdist
  distances =pdist(coords)
  print(f" Min bond distance: {distances.min():.2f}Å")
  print(f" Max bond distance: {distances.max():.2f}Å")
  print(f" {' Resonable' if 0.5 < distances.min() <2.0 else' check'}")
else:
  print(f" extract failed")


TESTING MACE ON QM9 MOLECULE

1. Molecule from QM9:
 SMILES:[H]C([H])([H])C([H])([H])C([H])([H])C(=O)C([H])(C([H])([H])[H])C([H])([H])[H]
 Number of atoms: 22
 true energy:0.659kcal/mol

2.Extracting 3d coordinates...
 success
 Shape:(22, 3)
Coordinates:
[[-3.44978282e+00  3.53899256e-01 -4.57180249e-02]
 [-2.19731366e+00 -4.27865431e-01 -4.06829231e-01]
 [-9.31993858e-01  3.28678293e-01 -1.17366111e-02]
 [ 3.37571140e-01 -4.29513842e-01 -3.72732045e-01]
 [ 3.07742719e-01 -1.55122903e+00 -8.78924599e-01]
 [ 1.65609413e+00  2.67866387e-01 -4.34946094e-02]
 [ 2.76373954e+00 -1.85303611e-01 -9.90752157e-01]
 [ 2.03394119e+00 -4.08485774e-03  1.40749431e+00]
 [-4.34364096e+00 -2.06481405e-01 -3.37084127e-01]
 [-3.50035412e+00  5.37568642e-01  1.03226437e+00]
 [-3.47149290e+00  1.31968503e+00 -5.60613932e-01]
 [-2.19316942e+00 -6.26567502e-01 -1.48523550e+00]
 [-2.22249792e+00 -1.40255137e+00  9.49696163e-02]
 [-9.03745772e-01  1.29568752e+00 -5.26154512e-01]
 [-9.28993203e-01  5.07636044e-

In [ ]:
# Make sure all dependencies are defined
print("Checking dependencies...")

# 1. Check MACE class exists
try:
    test_mace = MACE(hidden_dim=64, num_interactions=3)
    print(" MACE class defined")
except NameError:
    print(" MACE class not defined! Run the MACE class cell first")

# 2. Check MACELoss exists
try:
    test_loss = MACELoss()
    print(" MACELoss class defined")
except NameError:
    print(" MACELoss class not defined! Run the MACELoss cell first")

# 3. Check build_distance_edges exists
try:
    test_pos = torch.randn(5, 3)
    test_edges = build_distance_edges(test_pos)
    print("build_distance_edges function defined")
except NameError:
    print(" build_distance_edges not defined! Run that cell first")

# 4. Check get_molecule_coordinates exists
try:
    test_coords = get_molecule_coordinates("C")
    print(" get_molecule_coordinates function defined")
except NameError:
    print(" get_molecule_coordinates not defined! Need to define it")

print("\n All dependencies checked!")

Checking dependencies...
 MACE class defined
 MACELoss class defined
build_distance_edges function defined
 get_molecule_coordinates function defined

 All dependencies checked!


In [ ]:
# Testing on updated model - WITH DEBUG
print("="*70)
print("Testing updated Macemodel")
print("="*70)
import deepchem as dc

# Load small test dataset
tasks, datasets, _ = dc.molnet.load_qm9(
    featurizer="GraphConv",
    reload=True,
    splitter='random'
)
train_full, valid_full, test_full = datasets
print(f"Loaded: {len(train_full)}")

test_y = train_full.y[:5, 0]
test_w = train_full.w[:5, 0:1]
test_data = dc.data.NumpyDataset(
    X=train_full.X[:5],
    y=test_y,
    w=test_w,
    ids=train_full.ids[:5]
)
print(f" Test dataset: {len(test_data)} molecules")
print(f"   First SMILES: {test_data.ids[0]}")

# Populate coords_cache
print(f"\n3. Populating coords_cache...")
coords_cache = {}
for smiles_id in test_data.ids:
    coords = get_molecule_coordinates(smiles_id)
    if coords is not None:
        coords_cache[smiles_id] = coords
        print(f"   {smiles_id[:30]}... -> {coords.shape}")
    else:
        print(f"    Warning: Could not generate coordinates for {smiles_id}")

print(f"\n   Total in cache: {len(coords_cache)}")

# Create model with debug info
print(f"\n4. Creating SimpleMACEModel...")
print(f"   coords_cache type: {type(coords_cache)}")
print(f"   coords_cache size: {len(coords_cache)}")

try:
    model = SimpleMACEModel(coords_cache=coords_cache, batch_size=5)
    print(" Model created successfully!")
except Exception as e:
    print(f" Error creating model: {e}")
    import traceback
    traceback.print_exc()
    raise

# Test prediction
print(f"\n5. Testing prediction...")
predictions = model.predict(test_data)
print(f" Success!")
print(f"   Predictions shape: {predictions.shape}")
print(f"   Sample predictions: {predictions[:3]}")

# Quick training test
print(f"\n6. Quick training test (2 epochs)")
model.fit(test_data, nb_epoch=2)
print(f"\n Real coordinate extraction working in training!")

Testing updated Macemodel
✅ Loaded: 105576
✅ Test dataset: 5 molecules
   First SMILES: [H]C([H])([H])C([H])([H])C([H])([H])C(=O)C([H])(C([H])([H])[H])C([H])([H])[H]

3. Populating coords_cache...
   ✅ [H]C([H])([H])C([H])([H])C([H]... -> (22, 3)
   ✅ [H]O[C@@]1([H])[C@@]([H])(C([H... -> (23, 3)
   ✅ [H]N1C(=O)[C@@]2([H])[C@@]([H]... -> (15, 3)
   ✅ [H]c1c(C([H])(C([H])([H])[H])C... -> (19, 3)
   ❌ Warning: Could not generate coordinates for [H]C#C[C@]1(C#N)[N@H+]2C([H])([H])[C@]1([H])C2([H])[H]

   Total in cache: 4

4. Creating SimpleMACEModel...
   coords_cache type: <class 'dict'>
   coords_cache size: 4
✅ Model created successfully!

5. Testing prediction...
✅ Success!
   Predictions shape: (5,)
   Sample predictions: [24.47419  24.312737 11.664807]

6. Quick training test (2 epochs)
🔥 Using CUSTOM fit method (CPU mode)
Epoch 1/2, Loss: 28.8393, Time: 0.2s
✅ Training complete

✅ Real coordinate extraction working in training!


In [ ]:

# Check GPU
import torch
print(f"\n GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# Imports
import deepchem as dc
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem import AllChem
import pickle
import os

print("\n All imports ready!")
print("="*70)


✅ GPU available: False

✅ All imports ready!


In [ ]:
# prepare dataset
print(" scale up  using 5000 molecules")

import deepchem as dc
import time

print("\n1. Loading QM9 dataset..")
tasks, datasets,_=dc.molnet.load_qm9(
    featurizer ="GraphConv",
    splitter='random',
    reload=True
)

train_full, valid_full , test_full =datasets

print(f" Total available")
print(f" Training :{len(train_full)} molecules")
print(f" Validation:{len(valid_full)} molecules")

print("\n2. Preparing datasets for single task (energy)...")

train_size =5000
valid_size =1000 # Define a size for the validation set

train_y =train_full.y[: train_size, 0]
train_w =train_full.w[: train_size, 0:1]

valid_y =valid_full.y[: valid_size, 0] # Extract labels for validation
valid_w =valid_full.w[: valid_size, 0:1] # Extract weights for validation

train_data =dc.data.NumpyDataset(
    X=train_full.X[:train_size],
    y=train_y,
    w=train_w,
    ids=train_full.ids[:train_size]
)

valid_data =dc.data.NumpyDataset( # Create valid_data
    X=valid_full.X[:valid_size],
    y=valid_y,
    w=valid_w,
    ids=valid_full.ids[:valid_size]
)

print(f" prepared:")
print(f" training:{ len(train_data)} molecules")
print(f" validation:{len(valid_data)}molecules")

print("\n3. Dataset statistics:")
print(f" Energy range (training):")
print(f" Min:{train_y.min():.2f} kcal/mol")
print(f" mean:{train_y.mean():.2f} kcal/mol")

print("\n dataset preparation complete")

 scale up  using 5000 molecules

1. Loading QM9 dataset..
 Total available
 Training :105576 molecules
 Validation:13197 molecules

2. Preparing datasets for single task (energy)...
 prepared:
 training:5000 molecules
 validation:1000molecules

3. Dataset statistics:
 Energy range (training):
 Min:-1.65 kcal/mol
 mean:0.01 kcal/mol

 dataset preparation complete


In [ ]:
print("="*70)
print("PRE-COMPUTING COORDINATES")
print("="*70)

from tqdm import tqdm

# Load QM9
tasks, datasets, _ = dc.molnet.load_qm9(featurizer='GraphConv', splitter='random', reload=True)
train_full = datasets[0]

# Pre-compute
coords_cache = {}
failed = []

print("\nExtracting coordinates for 5000 molecules...")
for i in tqdm(range(5000)):
    smiles = train_full.ids[i]
    coords = get_molecule_coordinates(smiles)

    if coords is not None:
        coords_cache[smiles] = coords
    else:
        failed.append(smiles)

print(f"\n Success: {len(coords_cache)} molecules")
print(f" Failed: {len(failed)} molecules")

# Save
with open('coords_5k.pkl', 'wb') as f:
    pickle.dump(coords_cache, f)

print(" Coordinates cached!")
print("="*70)

PRE-COMPUTING COORDINATES

Extracting coordinates for 5000 molecules...


100%|██████████| 5000/5000 [06:07<00:00, 13.61it/s]


✅ Success: 4781 molecules
⚠️ Failed: 219 molecules
✅ Coordinates cached!


In [ ]:

print("Improve version of mace model")

print("\n1. Model configuration:")

config = {
    'batch_size': 4,  # Keep small for stability
    'learning_rate': 0.0005
}

print(f" Batch size: {config['batch_size']}")
print(f" Learning rate: {config['learning_rate']}")

print("\n2. creating macemodel....")

mace_model = SimpleMACEModel(
    coords_cache=coords_cache,  # ← ADD THIS FIRST!
    batch_size=config['batch_size'],
    learning_rate=config['learning_rate']
)

print("  model created")

total_params = sum(p.numel() for p in mace_model.model.parameters())
print(f"\n3. Model size")
print(f" Total parameters: {total_params:,}")

print("\n ready for training")

Improve version of mace model

1. Model configuration:
 Batch size: 4
 Learning rate: 0.0005

2. creating macemodel....
  model created

3. Model size
 Total parameters: 86,665

 ready for training


In [ ]:
print("\n Pre-training checks...")
print(f"   Train data: {len(train_data)} samples")
print(f"   Valid data: {len(valid_data)} samples")
print(f"   Train y shape: {train_data.y.shape}")
print(f"   Valid y shape: {valid_data.y.shape}")
print(f"   Coords cache size: {len(coords_cache)}")
print(f"   Model device: {next(mace_model.model.parameters()).device}")
print(" All checks passed!\n")


 Pre-training checks...
   Train data: 5000 samples
   Valid data: 1000 samples
   Train y shape: (5000,)
   Valid y shape: (1000,)
   Coords cache size: 4781
   Model device: cpu
 All checks passed!



In [ ]:
print("="*70)
print("🚀 DAY 2 FINAL TRAINING - 5K MOLECULES")
print("="*70)

import time
import numpy as np

# Pre-training checks
print("\n🔍 Pre-training checks...")
print(f"   Train data: {len(train_data)} samples")
print(f"   Valid data: {len(valid_data)} samples")
print(f"   Coords cache: {len(coords_cache)} molecules")
print(f"   Model device: {next(mace_model.model.parameters()).device}")
print("✅ Checks passed!")

# Training
print("\nStarting training (30 epochs)...")
print("=" * 70)

start_time = time.time()
mace_model.fit(train_data, nb_epoch=30)
training_time = time.time() - start_time

print(f"\n✅ Training complete: {training_time/60:.1f} minutes")

# Evaluation
print("\n📊 Evaluating on validation set...")
valid_predictions = mace_model.predict(valid_data)
valid_y = valid_data.y
day2_mae = np.mean(np.abs(valid_predictions.flatten() - valid_y.flatten()))

print("\n" + "="*70)
print("DAY 2 RESULTS")
print("="*70)
print(f"\n📊 Validation MAE: {day2_mae:.3f} kcal/mol")
print(f"⏱️  Training time: {training_time/60:.1f} minutes")
print(f"\n💡 Comparison:")
print(f"   Day 1: 0.705 kcal/mol")
print(f"   Day 2: {day2_mae:.3f} kcal/mol")

if day2_mae < 0.705:
    improvement = ((0.705 - day2_mae) / 0.705) * 100
    print(f"   ✅ Improvement: {improvement:.1f}%!")

print("\n🎉 DAY 2 COMPLETE!")
print("="*70)

🚀 DAY 2 FINAL TRAINING - 5K MOLECULES

🔍 Pre-training checks...
   Train data: 5000 samples
   Valid data: 1000 samples
   Coords cache: 4781 molecules
   Model device: cpu
✅ Checks passed!

Starting training (30 epochs)...
🔥 Using CUSTOM fit method (CPU mode)
Epoch 1/30, Loss: 1.2456, Time: 51.2s
Epoch 5/30, Loss: 1.0602, Time: 51.2s
Epoch 10/30, Loss: 1.0504, Time: 52.9s
Epoch 15/30, Loss: 1.0454, Time: 51.6s
Epoch 20/30, Loss: 1.0439, Time: 51.0s
Epoch 25/30, Loss: 1.0414, Time: 51.3s
Epoch 30/30, Loss: 1.0409, Time: 51.9s
✅ Training complete

✅ Training complete: 25.7 minutes

📊 Evaluating on validation set...

DAY 2 RESULTS

📊 Validation MAE: 0.811 kcal/mol
⏱️  Training time: 25.7 minutes

💡 Comparison:
   Day 1: 0.705 kcal/mol
   Day 2: 0.811 kcal/mol

🎉 DAY 2 COMPLETE!


In [ ]:
print("🔍 CHECKING DATASET FOR INVALID VALUES...")

# Check train_data
print("\n1. Train data:")
print(f"   X type: {type(train_data.X)}")
print(f"   y shape: {train_data.y.shape}")
print(f"   y has NaN: {np.isnan(train_data.y).any()}")
print(f"   y has Inf: {np.isinf(train_data.y).any()}")
print(f"   y min/max: {train_data.y.min():.2f} / {train_data.y.max():.2f}")

# Check a sample molecule
sample_graph = train_data.X[0]
sample_features = sample_graph.get_atom_features()
print(f"\n2. Sample molecule atomic numbers:")
print(f"   Raw values: {sample_features[:, 0]}")
print(f"   Min/Max: {sample_features[:, 0].min()} / {sample_features[:, 0].max()}")
print(f"   Has negative: {(sample_features[:, 0] < 0).any()}")
print(f"   Has NaN: {np.isnan(sample_features[:, 0]).any()}")

# Check coords_cache
print(f"\n3. Coords cache:")
sample_coords = list(coords_cache.values())[0]
print(f"   Sample shape: {sample_coords.shape}")
print(f"   Has NaN: {np.isnan(sample_coords).any()}")
print(f"   Has Inf: {np.isinf(sample_coords).any()}")

print("\n Diagnostic complete!")

🔍 CHECKING DATASET FOR INVALID VALUES...

1. Train data:
   X type: <class 'numpy.ndarray'>
   y shape: (5000,)
   y has NaN: False
   y has Inf: False
   y min/max: -1.65 / 10.18

2. Sample molecule atomic numbers:
   Raw values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
   Min/Max: 0.0 / 1.0
   Has negative: False
   Has NaN: False

3. Coords cache:
   Sample shape: (14, 3)
   Has NaN: False
   Has Inf: False

✅ Diagnostic complete!
